### Описание задания:
Решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно, применяя на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. 

### Этапы работы:

1) Получите данные и загрузите их в рабочую среду.

2) Подготовьте датасет к обучению моделей:
**a)** Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder.

3) Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

4) Обучите модель логистической регрессии с параметрами по умолчанию.

5) Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])

6) Оптимизируйте 3-4 параметра модели:
**a)** Используйте GridSearchCV.
**b)** Используйте RandomizedSearchCV.
**c)** Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
**d)** Повторите п. 5 после каждого итогового изменения параметров.

7) Сформулируйте выводы по проделанной работе:
**a)** Сравните метрики построенных моделей.
**b)** Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

In [1]:
import warnings
import numpy as np
import pandas as pd

from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [2]:
data = pd.read_csv('heart.csv')

In [3]:
data.duplicated().sum()

0

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [5]:
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


### Преобразование категориальных признаков

In [6]:
categorials = list(data.dtypes[data.dtypes == object].index)
dummy = pd.get_dummies(data[categorials], columns=categorials)

In [7]:
data = pd.concat([dummy, data.drop(columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])], axis=1)
data.head()

,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,...,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
0,0,1,0,1,0,0,0,1,0,1,...,0,0,1,40,140,289,0,172,0.0,0
1,1,0,0,0,1,0,0,1,0,1,...,0,1,0,49,160,180,0,156,1.0,1
2,0,1,0,1,0,0,0,0,1,1,...,0,0,1,37,130,283,0,98,0.0,0
3,1,0,1,0,0,0,0,1,0,0,...,0,1,0,48,138,214,0,108,1.5,1
4,0,1,0,0,1,0,0,1,0,1,...,0,0,1,54,150,195,0,122,0.0,0


In [8]:
X = data[data.columns[:-1]]
y = data['HeartDisease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21)

In [10]:
warnings.filterwarnings('ignore')

### Обучение модели логистической регрессии с параметрами по умолчанию.

In [11]:
lg_model = LogisticRegression()
lg_model.fit(X_train, y_train)

LogisticRegression()

In [12]:
score = cross_validate(lg_model, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score_lr = {(key, values.mean()) for (key, values) in score.items()}
mean_score_lr

{('fit_time', 0.05168209075927734),
 ('score_time', 0.013825321197509765),
 ('test_accuracy', 0.85382226469183),
 ('test_f1', 0.8669064541432734),
 ('test_precision', 0.8690723022267267),
 ('test_recall', 0.8734509803921569)}

### Оптимизация параметров модели

In [13]:
parameters = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'penalty': ['elasticnet', 'l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}

**(a) Используя GridSearchCV**

In [14]:
grid_search = GridSearchCV(
    estimator = lg_model,  
    param_grid = parameters,
    scoring = 'accuracy',
    cv = 10,
    n_jobs=-1,
)

In [15]:
grid_search.fit(X, y)

GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['elasticnet', 'l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [16]:
print(grid_search.best_params_)

{'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}


In [17]:
lg_model_grid = LogisticRegression(C=100, penalty='l2', solver='lbfgs')
lg_model_grid.fit(X_train, y_train)

score = cross_validate(lg_model_grid, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score_grid = {(key, values.mean()) for (key, values) in score.items()}
mean_score_grid

{('fit_time', 0.049213528633117676),
 ('score_time', 0.01250596046447754),
 ('test_accuracy', 0.85382226469183),
 ('test_f1', 0.8672199651657376),
 ('test_precision', 0.8680386132162967),
 ('test_recall', 0.8754117647058823)}

**(b) Используя RandomizedSearchCV**

In [29]:
rand = RandomizedSearchCV(lg_model, parameters)
rand.fit(X, y)

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                                        'penalty': ['elasticnet', 'l1', 'l2'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']})

In [30]:
print(rand.best_params_)

{'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


In [31]:
lg_model_rand = LogisticRegression(C=0.1, penalty='l2', solver='lbfgs')
lg_model_rand.fit(X_train, y_train)

score = cross_validate(lg_model_rand, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score_rand = {(key, values.mean()) for (key, values) in score.items()}
print('Усреднённые метрики качества модели LogisticRegression с подбором параметров RandomizedSearchCV:')
mean_score_rand

Усреднённые метрики качества модели LogisticRegression с подбором параметров RandomizedSearchCV:


{('fit_time', 0.03456456661224365),
 ('score_time', 0.00865178108215332),
 ('test_accuracy', 0.8505852842809365),
 ('test_f1', 0.8632695253562896),
 ('test_precision', 0.8657481503531601),
 ('test_recall', 0.8696078431372548)}

**(c) Добавление моделей классификации и вариации их параметров**

In [22]:
models=[
      {'name':'SVC',"model": SVC(), 'params':{'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto']}},
      {'name':'RandomForestClassifier','model': RandomForestClassifier(), 'params':{'n_estimators':[10,25,50,100,150,200], 'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}},
      {'name':'KNeighborsClassifier','model': KNeighborsClassifier(), 'params':{'n_neighbors':list(range(1,30)),'weights': ['uniform', 'distance'], 'p':[1,2,3]}},
      {'name':'DecisionTreeClassifier','model': DecisionTreeClassifier(), 'params':{'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}}
]
res = []
for value in  models:
    res.append((value['name'], RandomizedSearchCV(value['model'], value['params'], cv=10, n_jobs=-1).fit(X_train, y_train)))

In [23]:
for value in res:
    print(value[0], value[1].best_score_, value[1].best_params_)

SVC 0.8608848574601999 {'kernel': 'linear', 'gamma': 'scale'}
RandomForestClassifier 0.8676601258793039 {'n_estimators': 50, 'max_depth': 11, 'criterion': 'entropy'}
KNeighborsClassifier 0.7792299148463532 {'weights': 'distance', 'p': 1, 'n_neighbors': 20}
DecisionTreeClassifier 0.8569048500555349 {'max_depth': 3, 'criterion': 'entropy'}


**(d) Подсчёт метрик качества моделей из предыдущего шага по подобранным параметрам**

In [25]:
model = SVC(gamma='scale', kernel='linear')
model.fit(X_train, y_train)

score = cross_validate(model, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score = {(key, values.mean()) for (key, values) in score.items()}
print('Усреднённые метрики качества модели SVC:')
mean_score

Усреднённые метрики качества модели SVC:


{('fit_time', 5.022739362716675),
 ('score_time', 0.00947411060333252),
 ('test_accuracy', 0.8473124701385573),
 ('test_f1', 0.8614867276924996),
 ('test_precision', 0.8614013182066224),
 ('test_recall', 0.8676862745098038)}

In [26]:
model = RandomForestClassifier(n_estimators=50, max_depth=11, criterion='entropy')
model.fit(X_train, y_train)

score = cross_validate(model, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score = {(key, values.mean()) for (key, values) in score.items()}
print('Усреднённые метрики качества модели RandomForestClassifier:')
mean_score

Усреднённые метрики качества модели RandomForestClassifier:


{('fit_time', 0.13761866092681885),
 ('score_time', 0.016836166381835938),
 ('test_accuracy', 0.8637123745819398),
 ('test_f1', 0.8759431110889458),
 ('test_precision', 0.8707270796089377),
 ('test_recall', 0.8893725490196079)}

In [27]:
model = KNeighborsClassifier(weights='distance', p=1, n_neighbors=20)
model.fit(X_train, y_train)

score = cross_validate(model, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score = {(key, values.mean()) for (key, values) in score.items()}
print('Усреднённые метрики качества модели KNeighborsClassifier:')
mean_score

Усреднённые метрики качества модели KNeighborsClassifier:


{('fit_time', 0.005544710159301758),
 ('score_time', 0.08490159511566162),
 ('test_accuracy', 0.7667224080267558),
 ('test_f1', 0.7826217612256652),
 ('test_precision', 0.7926888708911021),
 ('test_recall', 0.7982745098039217)}

In [28]:
model = DecisionTreeClassifier(max_depth=3, criterion='entropy')
model.fit(X_train, y_train)

score = cross_validate(model, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])
mean_score = {(key, values.mean()) for (key, values) in score.items()}
print('Усреднённые метрики качества модели DecisionTreeClassifier:')
mean_score

Усреднённые метрики качества модели DecisionTreeClassifier:


{('fit_time', 0.004538273811340332),
 ('score_time', 0.008486223220825196),
 ('test_accuracy', 0.811359292881032),
 ('test_f1', 0.8251339727665273),
 ('test_precision', 0.8416233986663825),
 ('test_recall', 0.8202745098039216)}

### Вывод:

Было произведено сравнение значений метрики cross_validate для обученной модели LogisticRegression с параметрами по умолчанию с значения моделей, где параметры были оптимизированы с помощью GridSearchCV и RandomizedSearchCV.
Подбор параметров показал сравнимое качество моделью с параметрами по-умолчанию. Это обусловлено тем, что GridSearchCV и RandomizedSearchCV подобрали параметры почти аналогичные тем, что используются по-умолчанию.
При примерно одинаковых значениях метрики для двух моделей подбора параметров RandomizedSearchCV показал более высокую скорость подбора.

Также была проведена автоматическая подборка параметров для четырёх моделей классификации. Время перебора на локальной машине составило примерно 3 часа. Для ни также были посчитаны метрики с помощью cross_validate.
Самый лучший результат показала модель RandomForestClassifier, превзойдя качество LogisticRegression

Сравнивая метрики качества с полученные в проекте [Ensemble_methods](https://github.com/msavilov/Feature_Engineering_ML/blob/main/9_Ensemble_methods/ensemble_methods.ipynb), где применялись модели RandomForestClassifier и DecisionTreeClassifier, можно сделать вывод о том, что подбором параметров нам удалось немного улучшить качество моделей
